// Databricks notebook source exported at Wed, 10 Feb 2016 20:42:12 UTC


#![Wikipedia Logo](http://sameerf-dbc-labs.s3-website-us-west-2.amazonaws.com/data/wikipedia/images/w_logo_for_labs.png)

# Explore English Wikipedia pageviews by second
### Time to complete: 15 minutes

#### Business questions:

* Question # 1) How many rows in the table refer to *mobile* vs *desktop* site requests?
* Question # 2) How many total incoming requests were to the *mobile* site vs the *desktop* site?
* Question # 3) What is the start and end range of time for the pageviews data? How many days total of data do we have?
* Question # 4) Which day of the week does Wikipedia get the most traffic?
* Question #  5) Can you visualize both the mobile and desktop site requests together in a line chart to compare traffic between both sites by day of the week?

#### Technical Accomplishments:

* Use Spark's Scala and Python APIs
* Learn what a `sqlContext` is and how to use it
* Load a 255 MB tab separated file into a DataFrame
* Cache a DataFrame into memory
* Run some DataFrame transformations and actions to create visualizations
* Learn the following DataFrame operations: `show()`, `printSchema()`, `orderBy()`, `filter()`, `groupBy()`, `cast()`, `alias()`, `distinct()`, `count()`, `sum()`, `avg()`, `min()`, `max()`
* Write a User Defined Function (UDF)
* Join two DataFrames
* Bonus: Use Matplotlib and Python code within a Scala notebook to create a line chart



Dataset: http://datahub.io/en/dataset/english-wikipedia-pageviews-by-second

### Introduction to running Scala in Databricks Notebooks

Place your cursor inside the cells below, one at a time, and hit "Shift" + "Enter" to execute the code:

In [1]:
// This is a Scala cell. You can run normal Scala code here...
val x = 1 + 7

In [2]:
// Here is another Scala cell, that adds 2 to x
val y = 2 + x

In [3]:
// This line uses string interpolation to prints what y is equal to...
println(s"y is equal to ${y}")

y is equal to 10


In [4]:
// You can import additional modules and use them
import java.util.Date
println(s"This was last run on: ${new Date}")

This was last run on: Fri Feb 12 19:44:21 PST 2016


### DataFrames
A `sqlContext` object is your entry point for working with structured data (rows and columns) in Spark.

Let's use the `sqlContext` to read a table of the English Wikipedia pageviews per second.

In [5]:
// Notice that the sqlContext in Databricks is actually a HiveContext
sqlContext

org.apache.spark.sql.hive.HiveContext = org.apache.spark.sql.hive.HiveContext@4930213b

 A `HiveContext` includes additional features like the ability to write queries using the more complete HiveQL parser, access to Hive UDFs, and the ability to read data from Hive tables. In general, you should always aim to use the `HiveContext` over the more limited `sqlContext`.

Create a DataFrame named `pageviewsDF` and understand its schema:

In [8]:
// Note that we have pre-loaded the pageviews_by_second data into Databricks.
// You just have to read the existing table.

val pageviewsDF = sqlContext.read.table("pageviews_by_second")

Name: org.apache.spark.sql.catalyst.analysis.NoSuchTableException
Message: null
StackTrace: org.apache.spark.sql.hive.client.ClientInterface$$anonfun$getTable$1.apply(ClientInterface.scala:112)
org.apache.spark.sql.hive.client.ClientInterface$$anonfun$getTable$1.apply(ClientInterface.scala:112)
scala.Option.getOrElse(Option.scala:120)
org.apache.spark.sql.hive.client.ClientInterface$class.getTable(ClientInterface.scala:112)
org.apache.spark.sql.hive.client.ClientWrapper.getTable(ClientWrapper.scala:60)
org.apache.spark.sql.hive.HiveMetastoreCatalog.lookupRelation(HiveMetastoreCatalog.scala:227)
org.apache.spark.sql.hive.HiveContext$$anon$2.org$apache$spark$sql$catalyst$analysis$OverrideCatalog$$super$lookupRelation(HiveContext.scala:371)
org.apache.spark.sql.catalyst.analysis.OverrideCatalog$$anonfun$lookupRelation$3.apply(Catalog.scala:165)
org.apache.spark.sql.catalyst.analysis.OverrideCatalog$$anonfun$lookupRelation$3.apply(Catalog.scala:165)
scala.Option.getOrElse(Option.scala:120)

In [9]:
// Shows the first 20 records in ASCII print
pageviewsDF.show()

Name: Compile Error
Message: <console>:18: error: not found: value pageviewsDF
              pageviewsDF.show()
              ^
StackTrace: 

In [10]:
// The display() function also shows the DataFrame, but in a prettier HTML format (this only works in Databricks notebooks)

display(pageviewsDF)

Name: Compile Error
Message: <console>:19: error: not found: value display
              display(pageviewsDF)
              ^
StackTrace: 

 `printSchema()` prints out the schema, the data types and whether a column can be null:

In [11]:
pageviewsDF.printSchema()

Name: Compile Error
Message: <console>:17: error: not found: value pageviewsDF
              pageviewsDF.printSchema()
              ^
StackTrace: 

 Notice above that the first 2 columns are typed as `Strings`, while the requests column holds `Integers`. 

 Also notice, in a few cells above when we displayed the table, that the rows seem to be missing chunks of time.

The first row shows data from March 16, 2015 at **12:09:55am**, and the second row shows data from the same day at **12:10:39am**. There appears to be missing data between those time intervals because the original data file from Wikimedia contains the data out of order and Spark read it into a DataFrame in the same order as the file.

Our data set does actually contain 2 rows for every second (one row for mobile site requests and another for desktop site requests).

We can verify this by ordering the table by the timestamp column:

In [ ]:
// The following orders the rows by first the timestamp (ascending), then the site (descending) and then shows the first 10 rows

pageviewsDF.orderBy($"timestamp", $"site".desc).show(10)

### Reading from disk vs memory

The 255 MB pageviews file is currently on S3, which means each time you scan through it, your Spark cluster has to read the 255 MB of data remotely over the network.

In [ ]:
// Count how many total records (rows) there are
pageviewsDF.count()

 Hmm, that took about 10 - 20 seconds. Let's cache the DataFrame into memory to speed it up.

In [ ]:
sqlContext.cacheTable("pageviews_by_second")

Caching is a lazy operation (meaning it doesn't take effect until you call an action that needs to read all of the data). So let's call the `count()` action again:

In [ ]:
// During this count() action, the data is not only read from S3 and counted, but also cached
pageviewsDF.count()

The DataFrame should now be cached, let's run another `count()` to see the speed increase:

In [ ]:
pageviewsDF.count()

 Notice that operating on the DataFrame now takes less than 1 second!

### Exploring pageviews

Time to do some data analysis!

 
### Question #1:
**How many rows in the table refer to mobile vs desktop?**

In [ ]:
pageviewsDF.filter($"site" === "mobile").count()

In [ ]:
pageviewsDF.filter($"site" === "desktop").count()

 We can also group the data by the `site` column and then call count:

In [ ]:
pageviewsDF.groupBy($"site").count().show()

 So, 3.6 million rows refer to the mobile page views and 3.6 million rows refer to desktop page views.

 
### Question #2:
** How many total incoming requests were to the mobile site vs the desktop site?**

 First, let's sum up the `requests` column to see how many total requests are in the dataset.

In [ ]:
// Import the sql functions package, which includes statistical functions like sum, max, min, avg, etc.
import org.apache.spark.sql.functions._

In [ ]:
pageviewsDF.select(sum($"requests")).show()

 So, there are about 13.3 billion requests total.

 But how many of the requests were for the mobile site?

 ** Challenge 1:** Using just the commands we explored above, can you figure out how to filter the DataFrame for just mobile traffic and then sum the requests column?

In [ ]:
//Type in your answer here...
pageviewsDF.filter("site = 'mobile'").select(sum($"requests")).show()

 So, that many requests were for the mobile site (and probably came from mobile phone browsers).

 ** Challenge 2:** What about the desktop site? How many requests did it get?

In [ ]:
//Type in your answer here...
pageviewsDF.filter("site = 'desktop'").select(sum($"requests")).show()

 So, twice as many were for the desktop site.

 
### Question #3:
** What is the start and end range of time for the pageviews data? How many days of data do we have?**

 To accomplish this, we should first convert the `timestamp` column from a `String` type to a `Timestamp` type.

In [ ]:
// Currently in our DataFrame, `pageviewsDF`, the first column is typed as a string
pageviewsDF.printSchema()

 Create a new DataFrame, `pageviewsDF2`, that changes the timestamp column from a `string` data type to a `timestamp` data type.

In [ ]:
val pageviewsDF2 = pageviewsDF.select($"timestamp".cast("timestamp").alias("timestamp"), $"site", $"requests")

In [ ]:
pageviewsDF2.printSchema()

In [ ]:
display(pageviewsDF2)

 How many different years is our data from?

 For the next command, we'll use `year()`, one of the date time function available in Spark. You can review which functions are available for DataFrames in the [Spark API docs](https://spark.apache.org/docs/latest/api/scala/index.html#org.apache.spark.sql.functions$).

In [ ]:
pageviewsDF2.select(year($"timestamp")).distinct().show()

 The data only spans 2015. But which months?

 ** Challenge 3:** Can you figure out how to check which months of 2015 our data covers (using the Spark API docs linked to above)?

In [ ]:
//Type in your answer here...
pageviewsDF2.select(month($"timestamp")).distinct().show()

 The data covers the months you see above.

 ** Challenge 4:** How many weeks does our data cover?

*Hint, check out the Date time functions available in the  [Spark API docs](https://spark.apache.org/docs/latest/api/scala/index.html#org.apache.spark.sql.functions$).*

In [ ]:
//Type in your answer below...
pageviewsDF2.select(weekofyear($"timestamp")).distinct().show()

 The data set covers the number of weeks you see above. Similarly, we can see how many days of coverage we have:

In [ ]:
pageviewsDF2.select(dayofyear($"timestamp")).distinct().count()

 We have 41 days of data.

 To understand our data better, let's look at the average, minimum and maximum number of requests received for mobile, then desktop page views over every 1 second interval:

In [ ]:
// Look at mobile statistics
pageviewsDF2.filter("site = 'mobile'").select(avg($"requests"), min($"requests"), max($"requests")).show()

In [ ]:
// Look at desktop statistics
pageviewsDF2.filter("site = 'desktop'").select(avg($"requests"), min($"requests"), max($"requests")).show()

 There certainly appears to be more requests for the desktop site.

 
### Question #4:
** Which day of the week does Wikipedia get the most traffic?**

 Think about how we can accomplish this. We need to pull out the day of the week (like Mon, Tues, etc) from each row, and then sum up all of the requests by day.

 First, use the `date_format` function to extract out the day of the week from the timestamp and rename the column as "Day of week".

Then we'll sum up all of the requests for each day and show the results.

In [ ]:
// Notice the use of alias() to rename the new column
// "E" is a pattern in the SimpleDataFormat class in Java that extracts out the "Day in Week""

// Create a new DataFrame named pageviewsByDayOfWeekDF and cache it
val pageviewsByDayOfWeekDF = pageviewsDF2.groupBy(date_format(($"timestamp"), "E").alias("Day of week")).sum().cache()

// Show what is in the new DataFrame
pageviewsByDayOfWeekDF.show()

 You can learn more about patterns, like "E", that [Java SimpleDateFormat](https://docs.oracle.com/javase/7/docs/api/java/text/SimpleDateFormat.html) allows in the Java Docs.

 It would help to visualize the results:

In [ ]:
// This is the same command as above, except here we're tacking on an orderBy() to sort by day of week
display(pageviewsByDayOfWeekDF.orderBy($"Day of week"))

 Click on the Bar chart icon above to convert the table into a bar chart:

#![Bar Chart](http://i.imgur.com/myqoDNV.png)

 Also, under the Plot Options, you may need to set the Keys as "Day of week" and the values as "sum(requests)"

 Hmm, the ordering of the days of the week is off, because the `orderBy()` operation is ordering the days of the week alphabetically. Instead of that, let's start with Monday and end with Sunday. To accomplish this, we'll need to write a short User Defined Function (UDF) to prepend each `Day of week` with a number.

### User Defined Functions

A UDF lets you code your own logic for processing column values during a DataFrame query. 

First, let's create a Scala match expression for pattern matching:

In [ ]:
def matchDayOfWeek(day:String): String = {
  day match {
    case "Mon" => "1-Mon"
    case "Tue" => "2-Tue"
    case "Wed" => "3-Wed"
    case "Thu" => "4-Thu"
    case "Fri" => "5-Fri"
    case "Sat" => "6-Sat"
    case "Sun" => "7-Sun"
    case _ => "UNKNOWN"
  }
}

 Test the match expression:

In [ ]:
matchDayOfWeek("Tue")

 Great, it works! Now define a UDF named `prependNumber`:

In [ ]:
val prependNumberUDF = sqlContext.udf.register("prependNumber", (s: String) => matchDayOfWeek(s))

In [ ]:
// Note, here is a more idomatic Scala way of registering the same UDF
// val prependNumberUDF = sqlContext.udf.register("prependNumber", matchDayOfWeek _)

 Test the UDF to prepend the `Day of Week` column in the DataFrame with a number:

In [ ]:
pageviewsByDayOfWeekDF.select(prependNumberUDF($"Day of week")).show(7)

 Our UDF looks like it's working. Next, let's apply the UDF and also order the x axis from Mon -> Sun:

In [ ]:
display((pageviewsByDayOfWeekDF.withColumnRenamed("sum(requests)", "total requests")
  .select(prependNumberUDF($"Day of week"), $"total requests")
  .orderBy("UDF(Day of week)")))

 Click on the bar chart icon again to convert the above table into a Bar Chart. Also, under the Plot Options, you may need to set the Keys as "UDF(Day of week)" and the values as "total requests".

 Wikipedia seems to get significantly more traffic on Mondays than other days of the week.

 
### Question #5:
** Can you visualize both the mobile and desktop site requests in a line chart to compare traffic between both sites by day of the week?**

 First, graph the mobile site requests:

In [ ]:
val mobileViewsByDayOfWeekDF = pageviewsDF2.filter("site = 'mobile'").groupBy(date_format(($"timestamp"), "E").alias("Day of week")).sum().withColumnRenamed("sum(requests)", "total requests").select(prependNumberUDF($"Day of week"), $"total requests").orderBy("UDF(Day of week)").toDF("DOW", "mobile_requests")

// Cache this DataFrame
mobileViewsByDayOfWeekDF.cache()

display(mobileViewsByDayOfWeekDF)

 Click on the bar chart icon again to convert the above table into a Bar Chart. Also, under the Plot Options, you may need to set the Keys as "DOW" and the values as "mobile requests".

 Next, graph the desktop site requests:

In [ ]:
val desktopViewsByDayOfWeekDF = pageviewsDF2.filter("site = 'desktop'").groupBy(date_format(($"timestamp"), "E").alias("Day of week")).sum().withColumnRenamed("sum(requests)", "total requests").select(prependNumberUDF($"Day of week"), $"total requests").orderBy("UDF(Day of week)").toDF("DOW", "desktop_requests")

// Cache this DataFrame
desktopViewsByDayOfWeekDF.cache()

display(desktopViewsByDayOfWeekDF)

 Click on the bar chart icon to convert the above table into a Bar Chart.

 Now that we have two DataFrames (one for mobile views by day of week and another for desktop views), let's join both of them to create one line chart to visualize mobile vs. desktop page views:

In [ ]:
display(mobileViewsByDayOfWeekDF.join(desktopViewsByDayOfWeekDF, mobileViewsByDayOfWeekDF("DOW") === desktopViewsByDayOfWeekDF("DOW")))

 Click on the line chart icon above to convert the table into a line chart:

#![Line Chart](http://i.imgur.com/eXjxL5x.png)

 Then click on Plot Options:

#![Plot Options](http://i.imgur.com/sASSo9f.png)

 Finally customize the plot as seen below and click Apply:

*(You will have to drag and drop fields from the left pane into either Keys or Values)*

#![Customize Plot](http://i.imgur.com/VIyNNoA.png)

 Hmm, did you notice that the line chart is a bit deceptive? Beware that it looks like there were almost zero mobile site requests because the y-axis of the line graph starts from 600,000,000 instead of 0.

#![Customize Plot](http://i.imgur.com/YiThldl.png)

 Since the y-axis is off, it may appear as if there were almost zero mobile site requests. We can restore a zero baseline by using Matplotlib.

 
### Bonus:
** Use Matplotlib to fix the line chart visualization above so that the y-axis starts with 0 **

Use the Python notebook for this purpose